1 of 6: install correct Python version; setup virtual environment; install pip


In [ ]:
!apt-get update -y
!apt-get install python3.10 python3.10-distutils python3.10-venv -y


In [ ]:
!python3.10 -m venv /content/dmo_env


In [ ]:
%%bash
source /content/dmo_env/bin/activate

# Explicitly install pip within your existing venv
python -m ensurepip --upgrade

# Explicitly verify pip installation
which pip
pip --version


2 of 6: ensure GPU is available (e.g., runtime= "Connect T4") before cloning/downloading; show initial state of GPU

In [ ]:
# GPU check explicitly
import torch

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU is available: {device_name}")
else:
    print("❌ GPU is not available. Ensure GPU runtime is enabled.")


In [ ]:
!nvidia-smi

3 of 6: clone (fork of) DMOSpeech2

In [ ]:
import os

# Explicitly move to a known good directory
os.chdir('/content')

# Verify explicitly you're back in a good state
print("Current working directory explicitly set to:", os.getcwd())

!git clone https://github.com/wrightmikea/DMOSpeech2.git
%cd DMOSpeech2
!echo "pwd=`pwd`"

4 of 6: install requirements into activated virtual environment

In [ ]:

!source /content/dmo_env/bin/activate && pip install -r requirements.txt


5 of 6: fetch the DMOSpeech2 PyTorch model checkpoints from Huggingface into the cloned (forked) repo

In [ ]:
import os
os.chdir('/content/DMOSpeech2')

# Explicitly download checkpoints
!mkdir -p ckpts
!wget -O ckpts/model_85000.pt https://huggingface.co/yl4579/DMOSpeech2/resolve/main/model_85000.pt
!wget -O ckpts/model_1500.pt https://huggingface.co/yl4579/DMOSpeech2/resolve/main/model_1500.pt


6 of 6: verify environment; run the app to show the Gradio UI

In [ ]:
import subprocess

venv_path = "/content/dmo_env"
python_bin = f"{venv_path}/bin/python"

def run_command(cmd):
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE, text=True)
    return result.stdout.strip() if result.stdout else result.stderr.strip()

print("🔍 Explicit verification after corrected installation:")

# Verify PyTorch explicitly
torch_version = run_command(f'{python_bin} -c "import torch; print(torch.__version__)"')
print(f"• PyTorch version explicitly: {torch_version}")

# Verify torchaudio explicitly
torchaudio_version = run_command(f'{python_bin} -c "import torchaudio; print(torchaudio.__version__)"')
print(f"• torchaudio version explicitly: {torchaudio_version}")

# Verify CUDA explicitly
cuda_available = run_command(f'{python_bin} -c "import torch; print(torch.cuda.is_available())"')
print(f"• CUDA explicitly available: {cuda_available}")

# Explicitly confirm accelerate package explicitly installed
accelerate_version = run_command(f'{python_bin} -c "import accelerate; print(accelerate.__version__)"')
print(f"• accelerate explicitly installed: {accelerate_version}")


In [ ]:
!source /content/dmo_env/bin/activate && python --version && python scripts/colab-gradio.py

Python 3.10.12
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.114 seconds.
Prefix dict has been built successfully.
Word segmentation module jieba initialized.

Download Vocos from huggingface charactr/vocos-mel-24khz
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3b3209c351ede17406.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Converting audio...
Using custom reference text...

ref_text   this is a test. 
/content/dmo_env/lib/python3.10/site-packages/gradio/processing_utils.py:777: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


In [51]:
!curl http://localhost:7860/

curl: (7) Failed to connect to localhost port 7860 after 0 ms: Connection refused


In [60]:
!pwd
!ls
!grep -i launch scripts/*.*


/content/DMOSpeech2
ckpts	   data  LICENSE	     README.md	       scripts		   src
CLAUDE.md  info  original-README.md  requirements.txt  setup-source-me.sh
scripts/local-gradio.py:    demo.launch(server_name="127.0.0.1", server_port=7860)
scripts/remote-gradio.py:    demo.launch(server_name="0.0.0.0", server_port=7860)
